# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic
!pip install git+https://github.com/MaartenGr/BERTopic.git@refs/pull/668/merge
!pip install vaderSentiment

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data

In [18]:
from google.colab import drive
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk import FreqDist
from nltk import bigrams
from nltk.stem import SnowballStemmer

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
df=pd.read_excel('/content/drive/My Drive/Colab Notebooks/all_news.xlsx')

In [5]:
df.head()

,Unnamed: 0,Unnamed: 0.1,text,title,url,text_len,domain
0,5,4,Editor's Note — Monthly Ticket is a new CNN Tr...,"Faster, cleaner, greener: What lies ahead for ...",https://www.cnn.com/travel/article/future-rail...,14884,www.cnn.com
1,7,6,There are dozens of apps that may be needed to...,Inside the cumbersome world of EV charging and...,https://www.theglobeandmail.com/drive/mobility...,7478,www.theglobeandmail.com
2,11,10,Staying ahead of the technology curve means st...,5 Top Machine Learning Startups impacting Mobi...,https://www.startus-insights.com/innovators-gu...,5029,www.startus-insights.com
3,23,22,Kampala banks on technology intervention from ...,Kampala banks on technology intervention from ...,http://bit.ly/3463JqU,8724,http://bit.ly/3463JqU
4,27,26,"Like all of our readers, I’m pretty sad right ...",Clean Technology Activities Can Be A Small Par...,https://ift.tt/0gVkdoA,7183,ift.tt


In [6]:
df.shape

(18143, 7)

In [7]:
# TT
tmp0=pd.read_csv('/content/drive/My Drive/Colab Notebooks/idub_mk_tt/all0.csv')
tmp1=pd.read_csv('/content/drive/My Drive/Colab Notebooks/idub_mk_tt/all.csv')

In [8]:
tmp=pd.concat([tmp0,tmp1])
tmp=tmp.drop_duplicates(subset='url')
tmp=tmp[['text','created_at','umb','context_annotations','public_metrics','url']]

In [9]:
df=df.merge(tmp,on='url',suffixes=['','_tweet'])

In [10]:
df['text']=df['text'].str.lower()

In [25]:
# journals=['nyt','zdnet','forbes','guardian','wired','verge','technologyreview','cnet']

In [11]:
df['title']=df['title'].fillna('')
df['text_all']=df['title']+' '+df['text']

In [21]:
stopword_list=stopwords.words('english')

In [12]:
# df1=df[df['text_all'].str.contains('accessibility')]

In [13]:
keywords=['assistive technologies','assistive technology','accessibility','Accessibility','Assistive technologies','Assistive technology']

pattern = '|'.join(keywords)

df1=df[df['text_all'].str.contains(pattern)]

In [15]:
# pattern = '|'.join(journals)

# df1=df[df['domain'].str.contains(pattern)]

In [16]:
len(df1)

2645

In [ ]:
# df[df['text_title'].isna()]

In [ ]:
# docs=df1['text'].tolist()

In [17]:
df1.head()

,Unnamed: 0,Unnamed: 0.1,text,title,url,text_len,domain,text_tweet,created_at,umb,context_annotations,public_metrics,text_all
0,5,4,editor's note — monthly ticket is a new cnn tr...,"Faster, cleaner, greener: What lies ahead for ...",https://www.cnn.com/travel/article/future-rail...,14884,www.cnn.com,"Faster, cleaner, greener and packed with advan...",2022-05-31T15:43:00.000Z,all0,"[{'domain': {'id': '46', 'name': 'Brand Catego...","{'retweet_count': 19, 'reply_count': 3, 'like_...","Faster, cleaner, greener: What lies ahead for ..."
7,41,40,maurice flynn has spina bifida but it's not so...,Lack of accessible housing leads wheelchair-us...,https://www.stuff.co.nz/pou-tiaki/128744759/la...,5769,www.stuff.co.nz,"If you can’t buy one, BUILD one 🏡🧱🧰 #accessibi...",2022-05-30T09:55:59.000Z,all0,"[{'domain': {'id': '67', 'name': 'Interests an...","{'retweet_count': 3, 'reply_count': 0, 'like_c...",Lack of accessible housing leads wheelchair-us...
9,48,47,the importance of public purpose technology in...,The Importance of Public Purpose Technology in...,https://futuretransport-news.com/the-importanc...,6703,futuretransport-news.com,"With PPT, the transportation sector will benef...",2022-05-30T07:30:09.000Z,all0,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",The Importance of Public Purpose Technology in...
12,75,74,"\n\nnatasha layton, monash university; libby c...","From glasses to mobility scooters, ‘assistive ...",https://infotatile.com/from-glasses-to-mobilit...,6696,infotatile.com,"From glasses to mobility scooters, ‘assistive ...",2022-05-28T10:21:19.000Z,all0,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","From glasses to mobility scooters, ‘assistive ..."
13,77,76,olivia shivas is a pou tiaki reporter at stuff...,KiwiBuild is not built for all Kiwis,https://i.stuff.co.nz/pou-tiaki/128665590/kiwi...,5949,i.stuff.co.nz,"House-hunting, 😖. Wheelie house-hunting, 🤯! #a...",2022-05-28T09:08:57.000Z,all0,"[{'domain': {'id': '109', 'name': 'Reoccurring...","{'retweet_count': 2, 'reply_count': 0, 'like_c...",KiwiBuild is not built for all Kiwis olivia sh...


In [25]:
df1['text_token']=df1['text_all'].apply(lambda x: word_tokenize(x))
df1['text_token']=df1['text_token'].apply(lambda row: [i.lower() for i in row])
df1['text_token']=df1['text_token'].apply(lambda row: [i for i in row if i not in stopword_list and len(i)>1] )
# df['text_token_st']=df['text_token'].apply(lambda row: [st.stem(i) for i in row ] )

<ipython-input-25-16c13f496bba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['text_token']=df1['text_all'].apply(lambda x: word_tokenize(x))
<ipython-input-25-16c13f496bba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['text_token']=df1['text_token'].apply(lambda row: [i.lower() for i in row])
<ipython-input-25-16c13f496bba>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [26]:
docs=df1['text_token'].tolist()

In [ ]:
docs[0]

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [27]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

TypeError: ignored

**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [ ]:
len(docs)

2645

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,1047,-1_the_and_to_of
1,0,93,0_accessibility_website_ada_web
2,1,79,1_students_learning_education_and
3,2,79,2_cloud_migration_data_and
4,3,63,3_health_healthcare_patients_care


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('accessibility', 0.023615928468794192),
 ('website', 0.01993491183855124),
 ('ada', 0.01868625370328108),
 ('web', 0.017882059391246794),
 ('the', 0.016755916222179503),
 ('to', 0.01562213949029228),
 ('accessibe', 0.015538745611817251),
 ('wcag', 0.014782544988327118),
 ('websites', 0.014628470225617035),
 ('with', 0.013804017252630052)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [1]:
topic_model.visualize_barchart(top_n_topics=10)

NameError: ignored

In [ ]:
# topics_over_time = topic_model.topics_over_time(docs, df['Year'])
# topic_model.visualize_topics_over_time(topics_over_time,width=900)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
# topic_model.update_topics(docs, n_gram_range=(1, 2))

In [ ]:
# topic_model.get_topic(0)   # We select topic that we viewed before

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
# topic_model.reduce_topics(docs, nr_topics=60)

In [ ]:
# # Access the newly updated topics with:
# print(topic_model.topics_)

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("sustainability", top_n=5); similar_topics

NameError: ignored

In [ ]:
topic_model.get_topic(35)

[('market', 0.05869605353202775),
 ('green', 0.04532426918360536),
 ('sustainability', 0.027208155542827288),
 ('report', 0.022395717725805828),
 ('forecast', 0.01809013187917263),
 ('technology', 0.01790403581525162),
 ('analysis', 0.017359868006520247),
 ('exhibit', 0.016367960393030054),
 ('20212026', 0.015522962102278031),
 ('growth', 0.015514911844704098)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
